In [58]:
import pandas as pd
from kats.models.prophet import ProphetModel, ProphetParams
from kats.consts import TimeSeriesData
from kats.models.var import VARModel, VARParams
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'kats'

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [45 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib
      creating build\lib\fbprophet
      creating build\lib\fbprophet\stan_model
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Uporabnik\AppData\Local\Temp\pip-install-ykdaj4jg\fbprophet_a8eb172fc12f4f99a36ca84e7caf55ad\setup.py", line 122, in <module>
          setup(
        File "c:\Users\Uporabnik\AppData\Local\Programs\Python\Python39\lib\site-packages\setuptools\__init__.py", line 165, in setup
          return distutils.core.setup(**attrs)
        File "c:\Users\Uporabnik\AppData\Local\Programs\Python\Python39\lib\distutils\core.py", line 148, in setup
          dist.run_commands()
        File "c:\Users

  Using cached kats-0.2.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached numpy-1.21.6-cp39-cp39-win_amd64.whl.metadata (2.2 kB)
  Using cached pandas-1.3.5-cp39-cp39-win_amd64.whl.metadata (12 kB)
  Using cached pystan-2.19.1.1.tar.gz (16.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached scipy-1.7.3-cp39-cp39-win_amd64.whl.metadata (2.2 kB)
  Using cached setuptools_git-1.2-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached statsmodels-0.12.2-cp39-none-win_amd64.whl.metadata (8.3 kB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl.metadata (6.3 kB)
  Using cached ax_platform-0.2.4-py3-none-any.whl.metadata (9.0 kB)
  Using cached gpytorch-1.11-py3-none-any.whl.metadata (7.4 kB)
  Usin

In [57]:
pip install kats

^C
Note: you may need to restart the kernel to use updated packages.


In [6]:
## podatki, ki so nam jih poslali:
podatki = pd.read_csv(os.path.join('C:/Users/Uporabnik/OneDrive/Namizje/SOLA/MAGISTERIJ/MATEMATIKA Z RACUNALNIKOM/GospodinjskiOdjemalci.csv'), sep=";")
# Malo boljši format:
podatki['DateTimeStartUTC'] = pd.to_datetime(podatki['DateTimeStartUTC'], format='%d.%m.%Y %H:%M')
podatki['DateTimeStartCET'] = pd.to_datetime(podatki['DateTimeStartCET'], format='%d.%m.%Y %H:%M')

podatki = podatki[['DateTimeStartUTC', 'Odjem ACT', 'Temperatura ACT', 'Temperatura FC', 'Sevanje ACT', 'Sevanje FC']]

podatki.set_index('DateTimeStartUTC', inplace=True)
podatki['Year'] = podatki.index.year
podatki['Month'] = podatki.index.month
podatki['Day'] = podatki.index.date

podatki = podatki.asfreq('15T')

podatki['Odjem ACT'] = podatki['Odjem ACT'] * 1000000000 # da dobim MWh (bo lepše pri logaritmu tega stolpce v nadaljevanju)

podatki

,Odjem ACT,Temperatura ACT,Temperatura FC,Sevanje ACT,Sevanje FC,Year,Month,Day
DateTimeStartUTC,,,,,,,,
2021-10-31 23:00:00,9508.67,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-10-31 23:15:00,9168.37,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-10-31 23:30:00,8860.11,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-10-31 23:45:00,8510.97,3.60,5.300,0.0,0.0,2021,10,2021-10-31
2021-11-01 00:00:00,8332.21,3.45,5.300,0.0,0.0,2021,11,2021-11-01
...,...,...,...,...,...,...,...,...
2024-02-29 21:45:00,12005.50,6.80,7.475,0.0,0.0,2024,2,2024-02-29
2024-02-29 22:00:00,11336.40,6.65,7.300,0.0,0.0,2024,2,2024-02-29
2024-02-29 22:15:00,10796.70,6.50,7.300,0.0,0.0,2024,2,2024-02-29


In [ ]:
# multivariatni modeli

In [ ]:
# 1. LMulti-Variate Time-series Forecasting using Kats Model